In [1]:
import numpy as np
import pandas as pd
import psycopg2
import random
import glob
import os
import openpyxl
import re

In [2]:
def grab_all_workouts(client_name):
    file=openpyxl.load_workbook(f'/Users/jaredperez/Client Folders/{client_name}.xlsx')
    all_sheetnames = file.sheetnames
    valid_sheetnames = [name for name in all_sheetnames if "skip" not in name]
    sheets=[file[i] for i in valid_sheetnames]
    exercises=[]
    start_row = 0

    for block in sheets:
        for row in block.iter_rows(max_col=3):
            values = []
            for cell in row:
                values.append(cell.value)
                if "workout" in str(cell.value).lower():
                    start_row = cell.row
                if start_row > 0 and cell.row >= start_row and cell.row <= start_row + 12:
                    if len(values) == 3:
                        exercises.append((values[0], values[1], values[2]))
                
    workouts = []
    current_workout = []

    start = 0
    for idx, row in enumerate(exercises):
        if row[0] is None:
            continue
        if "workout" in str(row[0]).lower():
            if current_workout:
                workouts.append(current_workout)
            current_workout = []
            start = idx
        current_workout.append(row)
    stop = idx

    if current_workout:
        workouts.append(current_workout)
        
    dfs = [pd.DataFrame(i[1:], columns=['Exercise', 'Prescription', 'Weight']) for i in workouts]
    for df in dfs:
        df['Weight'] = df['Weight'].fillna(1)
        try:
            df['Weight']=df['Weight'].astype(dtype='float64')
        except:
            pass

    
    return dfs

In [3]:
def clean_workouts(list_of_dataframes):
    cleaned_dataframes = []
    for df in list_of_dataframes:
        if df['Weight'].dtype == 'float64':
            cleaned_dataframes.append(df)
            list_of_dataframes = cleaned_dataframes

            
    for df in list_of_dataframes:
        if df['Prescription'].apply(lambda x: isinstance(x, int)).any():
            df['Prescription'] = df['Prescription'].astype(str)

    prescriptions=[i['Prescription'] for i in list_of_dataframes]
    prescriptions=[i.fillna('1x') for i in prescriptions]
    
    pattern = r'^(\d+)x(\d+)?(?:\s*sec|\s*\(\s*\w*\s*\)|\s*min|\s*@|)?'
    outputs = []
    for arr in prescriptions:
        parts_full = []
        for s in arr:
            try:
                parts = re.findall(pattern, s)
                parts_full.append(parts)
            except:
                pass
        outputs.append(parts_full)
    
    outputs_fixed = [item for item in outputs if item != [[]]]
    for idx,i in enumerate(outputs_fixed):
        if any(not sub for sub in i):
            outputs_fixed[idx] = [[('1', '')] if not sub else sub for sub in i]
        
    sets_reps=[np.array(i).reshape(-1,2) for i in outputs_fixed]
    sets_reps=[np.where(i=='','1',i).astype(dtype='int') for i in sets_reps]
    
    volume=[i[:,0]*i[:,1] for i in sets_reps]
    
    for df,arr in zip(list_of_dataframes,volume):
        try:
            df['Volume-Load']=df['Weight']*arr
        except:
            pass
        
    for df in list_of_dataframes:
        try:
            df['Total'] = pd.Series([0] * len(df))
            df.loc[len(df) - 1, 'Total'] = df['Volume-Load'].sum()
        except:
            pass
        
    return list_of_dataframes

In [4]:
files=grab_all_workouts('Misha')

In [5]:
files=clean_workouts(files)

In [6]:
random.choice(files)

,Exercise,Prescription,Weight,Volume-Load,Total
0,Mountain Climbers*,3x30 sec,1.0,90.0,0
1,Plank to Pushup*,3x30 sec,1.0,90.0,0
2,Side Planks,3x45 Sec,1.0,135.0,0
3,Pushups,3x10,1.0,30.0,0
4,Alternative reverse Lunge,3x10 EL,1.0,30.0,0
5,SL RDL,3x5 ES,1.0,15.0,0
6,Bicycles,3x30 sec,1.0,90.0,0
7,Hollow Holds,3x30 sec,1.0,90.0,0
8,Stairs (every other step),x20,1.0,1.0,571


In [4]:
def intensity_classification(workouts):
    totals=[workouts['Total'].sum() for i in files]

for total in totals:
    if total<=manual_cuts[0].max():
        intensity='Light'
    if total<=manual_cuts[1].max() and total>=manual_cuts[0].max():
        intensity='Moderate-Light'
    if total<=manual_cuts[2].max() and total>=manual_cuts[1].max():
        intensity='Moderate' 
    if total<=manual_cuts[3].max() and total>=manual_cuts[2].max():
        intensity='Moderate-Heavy'
    if total<=manual_cuts[4].max() and total>=manual_cuts[3].max():
        intensity='Heavy'
        
    print(intensity)

NameError: name 'totals' is not defined